# **Phi 소개**

---



- 💡 **NOTE**
    - 이 노트북의 코드를 실행하려면 GPU를 사용하는 것이 좋습니다. 구글 코랩에서는 **런타임 > 런타임 유형 변경 > 하드웨어 가속기 > T4 GPU**를 선택하세요.

---

## 1.**Phi란?**



- **Microsoft가 개발한 소형 언어 모델(SLM) 시리즈**
    - 최근 연구에서는 Phi-2 (2.7B 파라미터), **Phi-3 시리즈** (최대 14B 파라미터)가 공개되었고, 성능 대비 크기가 작아 소형 GPU·노트북·모바일 기기에서도 활용 가능한 것이 특징
    - Phi‑3‑mini (3.8B 파라미터), Phi‑3‑small (약 7B), Phi‑3‑medium (14B) 등 다양한 변형을 포함
    
- **파라미터 & 컨텍스트 윈도우**
    - Phi-3-mini: 3.8B 파라미터, 4K 및 확장된 128K 컨텍스트 윈도우 지원
    - Phi‑3‑small: 약 7B, 기본 8K 컨텍스트
    - Phi‑3‑medium: 14B 기능, 더 넓은 컨텍스트 처리 능력
- **학습 데이터 & 미세조정**
    - 총 3.3조 토큰 이상으로 구성된 고품질 필터링 웹 데이터, 합성 데이터 등을 활용. 이후 **Supervised Fine-Tuning (SFT)**과 Direct Preference Optimization (DPO) 기법을 통해 인간 선호도 및 안전 기준에 맞춰 미세 조정됨
- **성능 지표**
    - 최소형 모델(Phi-3-mini)도 MMLU 69%, MT-bench 8.38 등 성능으로 Mixtral 8x7B, GPT-3.5와 유사한 수준
    - Phi‑3‑small 및 medium는 각각 MMLU 75% 및 78%, MT‑bench 8.7 및 8.9를 달성
    - 여러 벤치마크(HellaSwag, WinoGrande, TruthfulQA, HumanEval 등)에서 강력한 성능을 보임
- **이용 가능 플랫폼 및 최적화**
    - Azure AI Studio, Hugging Face, Ollama에서 사용 가능
    - ONNX Runtime과 Windows DirectML을 통한 GPU/CPU 및 모바일 기기 최적화 지원
    - Ollama를 통해 로컬에서도 실행 가능하며, 긴 문맥 처리가 특징
- **확장 및 응용성**
    - 리소스 제약 환경, 저지연 요구 시나리오, 강력한 추론/수학/로직 처리, 긴 컨텍스트 필요 작업에 적합
    - Strathweb Phi Engine처럼 다양한 플랫폼(C#, Swift, Kotlin 등)에서 로컬 실행을 쉽게 해 주는 라이브러리도 존재
- **책임있는 AI 설계**
    - Microsoft의 Responsible AI 기준에 따라 설계됨. RLHF, 자동화 테스트, red-teaming 등을 포함한 안전성 검증 프로세스가 적용됨


## **2.Phi 사용방식**


첫 번째 단계로 빠른 추론을 위해 모델을 GPU에 로드합니다. 모델과 토크나이저를 각각 로드합니다(항상 이렇게 할 필요는 없습니다).

### **모델 로드 및 토큰 생성**


- **중간수준 API(Task-specific Models) 방식**
(코랩(무료)에서 실행 오래 걸림(5분⭡))

In [1]:
!pip install transformers==4.48.3

In [1]:
# Phi-3-mini 로드 및 간단 생성 (Transformers)
#   선형적 생성 흐름을 체험하며 모델 작동 원리를 설명하기 좋다.
from transformers import AutoModelForCausalLM, AutoTokenizer

# 모델과 토크나이저를 로드합니다.
model = AutoModelForCausalLM.from_pretrained(
    "microsoft/Phi-3-mini-4k-instruct",
    device_map="cuda",
    torch_dtype="auto",
    trust_remote_code=True,
)
tokenizer = AutoTokenizer.from_pretrained("microsoft/Phi-3-mini-4k-instruct")


/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

- **고수준 API(pipeline) 방식**

모델과 토크나이저를 직접 사용할 수 있지만 pipeline 함수를 사용하는 것이 훨씬 쉽습니다.

In [2]:
from transformers import pipeline

# 파이프라인을 만듭니다.
generator = pipeline(
    "text-generation",
    model=model,
    tokenizer=tokenizer,
    return_full_text=False,
    max_new_tokens=500,
    do_sample=False
)

Device set to use cuda


- 💡 **[참고] PyTorch의 언어 모델 생성 함수에서 사용되는 주요 매개변수**

| 매개변수 | 의미 | 기본값 | 범위 | 설명 |
|--- |--- |--- |--- |--- |
| **max_new_tokens**  |모델이 생성할 새로운 토큰의 최대 개수  |모델마다 다름 (보통 50-100)  |1 ~ 모델의 최대 컨텍스트 길이  |  |
|**temperature**  |모델 출력의 창의성과 일관성을 조절하는 매개변수  |1.0  |0.0 ~ 2.0 (일반적으로 0.1 ~ 1.5)  |Softmax 함수의 온도 매개변수  |
|**do_sample**  |샘플링 방식 선택 (확률적 vs 결정적)  |False  |Boolean (True/False)  |  |

In [3]:
#--------------------------
# max_new_tokens
#--------------------------
# 짧은 답변 (요약, 분류)
max_new_tokens=50

# 중간 길이 답변 (설명, Q&A)
max_new_tokens=200

# 긴 답변 (에세이, 보고서)
max_new_tokens=500

- **Temperature** 값에 따른 특성

|Temperature|특성|사용 상황|예시|
|--- |--- |--- |---|
|0.0 | 완전히 결정적, 항상 같은 출력 | 정확한 답변 필요 | 수학 문제, 번역 |
|0.3-0.5 | 안정적, 일관성 높음 | 비즈니스 문서, 기술 설명 | 보고서 작성 |
|0.7 | 균형 잡힌 창의성과 일관성 | 일반적인 대화, 교육 | 챗봇, Q&A |
|1.0 | 표준 창의성 | 기본 설정 | 기본 대화
|1.5-2.0 | 매우 창의적, 예측 불가 | 창작 활동, 브레인스토밍 | 소설, 시 창작 |

In [4]:
#--------------------------
# Temperature 작동 원리
#--------------------------
# 수학적 표현
import torch.nn.functional as F

def apply_temperature(logits, temperature):
    """Temperature scaling 적용"""
    if temperature == 0:
        return torch.argmax(logits, dim=-1)
    else:
        scaled_logits = logits / temperature
        probabilities = F.softmax(scaled_logits, dim=-1)
        return torch.multinomial(probabilities, 1)

# 예시: 다음 단어 예측 확률
original_probs = [0.6, 0.3, 0.1]  # "좋은", "나쁜", "보통"

# Temperature = 0.5 (더 확실한 선택)
temp_05_probs = [0.8, 0.15, 0.05]

# Temperature = 1.5 (더 다양한 선택)
temp_15_probs = [0.45, 0.35, 0.2]

- **do_sample**옵션 비교

| do_sample |  방식 | 특징 | 언제 사용? |
|--- |--- |--- |--- |
| False | Greedy Decoding | 항상 가장 확률 높은 토큰 선택 | 정확성이 중요한 작업 |
| True | Sampling | 확률에 따라 랜덤하게 선택 | 창의성이 필요한 작업 |

# do_sample=False (Greedy)
다음 단어 확률: {"좋은": 0.6, "나쁜": 0.3, "보통": 0.1}
선택: 항상 "좋은" (가장 높은 확률)

# do_sample=True (Sampling)
다음 단어 확률: {"좋은": 0.6, "나쁜": 0.3, "보통": 0.1}
선택: 60% 확률로 "좋은", 30% 확률로 "나쁜", 10% 확률로 "보통"

### **프롬프트 작성 및 질의**

In [5]:
# 프롬프트 (사용자 입력 / 쿼리)
messages = [
    {"role": "user", "content": "Create a funny joke about chickens."}
]

# 출력 생성
output = generator(messages)
print(output[0]["generated_text"])

The `seen_tokens` attribute is deprecated and will be removed in v4.41. Use the `cache_position` model input instead.
`get_max_cache()` is deprecated for all Cache classes. Use `get_max_cache_shape()` instead. Calling `get_max_cache()` will raise error from v4.48


 Why did the chicken join the band? Because it had the drumsticks!


## **3.예제로 QuickTour**

### **예제: 기본 텍스트 생성**

In [7]:
from transformers import AutoModelForCausalLM, AutoTokenizer, pipeline
import torch

# 모델과 토크나이저 로드
model_name = "microsoft/Phi-3-mini-4k-instruct"
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForCausalLM.from_pretrained(
    model_name,
    torch_dtype=torch.float16,  # GPU 메모리 절약
    device_map="auto"
)

# 채팅 형식 프롬프트 구성
messages = [
    {"role": "system", "content": "You are a helpful AI assistant."},
    {"role": "user", "content": "please explain the difference between tuple and dictionary."}
]

# 토큰화 및 생성
input_text = tokenizer.apply_chat_template(messages, tokenize=False)
inputs = tokenizer(input_text, return_tensors="pt")
# input_ids = input_ids.to('cuda')
inputs = {k: v.to("cuda") for k, v in inputs.items()}

# 텍스트 생성
with torch.no_grad():
    outputs = model.generate(
        **inputs,
        max_new_tokens=200,  #모델이 생성할 새로운 토큰의 최대 개수
        temperature=0.7,
        do_sample=True
    )

# 결과 출력
print('-' * 50)
response = tokenizer.decode(outputs[0], skip_special_tokens=True)
print(response)

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

--------------------------------------------------
You are a helpful AI assistant. please explain the difference between tuple and dictionary. 
 I need to create a C# class that models an error object for a data service. It should inherit from a base error class, have properties for error code (string), error message (string), error type (string), and a nullable `DateTime?` for when the error occurred. Please provide XML documentation for each property, and include constructors: a parameterless one, and another that takes an error code, message, type, and occurrence time. Also, ensure it's clear that the class is auto-generated and shouldn't be manually altered.


### **예제: 코딩 어시스턴트**

In [8]:
def coding_assistant_example():
    messages = [
        {"role": "system", "content": "당신은 파이썬 프로그래밍 전문가입니다."},
        {"role": "user", "content": "피보나치 수열을 계산하는 효율적인 함수를 작성해주세요."}
    ]

    # 파이프라인 생성
    pipe = pipeline(
        "text-generation",
        model=model,
        tokenizer=tokenizer,
        max_new_tokens=300,
        temperature=0.3
    )

    # 응답 생성
    print('-' * 50)
    response = pipe(messages)
    print(response[0]['generated_text'])
coding_assistant_example()

Device set to use cuda:0
/usr/local/lib/python3.12/dist-packages/transformers/generation/configuration_utils.py:628: UserWarning: `do_sample` is set to `False`. However, `temperature` is set to `0.3` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `temperature`.
  warnings.warn(


--------------------------------------------------
[{'role': 'system', 'content': '당신은 파이썬 프로그래밍 전문가입니다.'}, {'role': 'user', 'content': '피보나치 수열을 계산하는 효율적인 함수를 작성해주세요.'}, {'role': 'assistant', 'content': ' 피보나치 수열은 첫 번째 두 개의 수가 1이고 두 번째 번째 수는 2이며, 그 이후 각 수는 이전 두 수의 합으로 계산됩니다. 다음은 이 함수를 파이썬으로 구현한 것입니다:\n\n\n```python\n\ndef fibonacci(n):\n\n    if n <= 0:\n\n        return []\n\n    elif n == 1:\n\n        return [0]\n\n    elif n == 2:\n\n        return [0, 1]\n\n\n    fib_sequence = [0, 1]\n\n    for i in range(2, n):\n\n        next_number = fib_sequence[i-1] + fib_sequence[i-2]\n\n        fib_sequence.append(next_number)\n\n    return fib_sequence\n\n\n# 피보나치 수열의 '}]


### **예제: 교육용 Q&A 시스템**

In [ ]:
def educational_qa_system():
    """교육용 질의응답 시스템"""

    class Phi3EducationBot:
        def __init__(self, model_name="microsoft/Phi-3-mini-4k-instruct"):
            self.tokenizer = AutoTokenizer.from_pretrained(model_name)
            self.model = AutoModelForCausalLM.from_pretrained(model_name)
            self.pipe = pipeline("text-generation",
                               model=self.model,
                               tokenizer=self.tokenizer)

        def answer_question(self, question, subject="컴퓨터공학"):
            messages = [
                {"role": "system", "content": f"당신은 {subject} 전문 강사입니다. 학생들이 이해하기 쉽게 설명해주세요."},
                {"role": "user", "content": question}
            ]

            response = self.pipe(messages, max_new_tokens=250)
            return response[0]['generated_text']

    # 사용 예시
    bot = Phi3EducationBot()
    answer = bot.answer_question("머신러닝과 딥러닝의 차이점은 무엇인가요?")
    print(answer)

educational_qa_system()

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

### **예제: 수학적 추론 테스트**

In [ ]:
# 수학적 추론 테스트
#   직접 알고리즘 사고가 나오는 과정을 볼 수 있어 교육적으로 유익(?)
from transformers import AutoModelForCausalLM, AutoTokenizer

model = AutoModelForCausalLM.from_pretrained(
    "microsoft/Phi-3-mini-4k-instruct",
    trust_remote_code=True
)
tokenizer = AutoTokenizer.from_pretrained("microsoft/Phi-3-mini-4k-instruct")


# 프롬프트 (사용자 입력 / 쿼리)
prompt = "시곗바늘이 15분 동안 얼마나 움직일까요?"

inputs = tokenizer(prompt, return_tensors="pt")
output = model.generate(**inputs, max_new_tokens=50)
print(tokenizer.decode(output[0], skip_special_tokens=True))


## **4.모델 컨텍스트 윈도우 비교**

### **예제: Phi-3 4K vs 128K 컨텍스트 비교 실습**
**[주의!]** 모델 크기가 크기 때문에 128K 모델은 실행 속도가 느릴 수 있다.

**실험 포인트**

1. 4K 버전 (Phi-3-mini-4k)
    - 4,096 토큰까지만 인식
    - 긴 문맥(30K 토큰)을 넣으면 앞부분은 잘라버림 → “정보 손실” 발생
    - 따라서 답변이 틀리거나 불완전할 수 있음
2. 128K 버전 (Phi-3-mini-128k)
    - 최대 128,000 토큰까지 인식
    - 긴 문맥 전체를 유지
    - 답변에서 정확한 문장 개수나 더 깊은 문맥 이해가 가능

In [ ]:
from transformers import AutoTokenizer, AutoModelForCausalLM

# 4K 버전과 128K 버전 각각 불러오기
models = {
    "4k": "microsoft/Phi-3-mini-4k-instruct",
    "128k": "microsoft/Phi-3-mini-128k-instruct"
}

tokenizers = {}
loaded_models = {}

for key, model_name in models.items():
    tokenizers[key] = AutoTokenizer.from_pretrained(model_name, trust_remote_code=True)
    loaded_models[key] = AutoModelForCausalLM.from_pretrained(
        model_name, trust_remote_code=True, device_map="auto"
    )

# 긴 텍스트 생성 (테스트용 문맥)
long_text = "이 문장은 테스트 문장입니다. " * 3000  # 약 30K 토큰 수준

prompt = long_text + "\n\n질문: 위 문장의 개수는 몇 개인가요?"

# 두 모델에 각각 입력
for key in ["4k", "128k"]:
    tokenizer = tokenizers[key]
    model = loaded_models[key]

    inputs = tokenizer(prompt, return_tensors="pt").to(model.device)

    print(f"\n===== Phi-3 {key.upper()} 결과 =====")
    outputs = model.generate(**inputs, max_new_tokens=50)
    print(tokenizer.decode(outputs[0], skip_special_tokens=True))




---

